In [86]:
#This cell has to be executed
load('superpartition.py')
load('sym_superfunct.py')

# Superpartition and symmetric superpolynomials

## Superpartitions
### Superpartition: the $(\Lambda^a; \Lambda^s)$ description. 
The superpartitions are introduced in the following way. 
#### `Superpartitions()` class
We first introduce the complete space of superpartitions like so

In [87]:
S = Superpartitions() # Creates an instance of the set of superpartitions
S #We print the description of the just created set. 

superpartitions

Let us consider a superpartition.
 (Note that since Python is underlying  Sage, one should never use `lambda` as variable name since it is a special function.)

In [88]:
my_spart = S([[3,2],[4,2,2]]) # This how you define a superpartition
my_spart # We print it

[[3, 2], [4, 2, 2]]

We know that for superpartition $\Lambda$ we have
$$
\Lambda_1 > \Lambda_2 > \cdots > \Lambda_m \geq 0 \quad \text{and} \quad
\Lambda_{m+1} \geq \Lambda_{m+2} \geq \cdots \geq \Lambda_{\ell} > 0
$$
This is enforced into the declaration of superpartitions. Trying to define a superpartition that violates these conditions will raise an error:

In [89]:
try: # This tries to execute the following statement
        my_bad_spart = S([[3,2,1,1],[5]]) # This generates an error
except ValueError as err: # If an error was raised, the following is executed
    print("The following error was raised: ", err)

('The following error was raised: ', ValueError('The fermionic partition must contain decreasing distinct parts >= 0',))


#### `len( a_superpartition )` and `sector()`
Back to our superpartition, we can retrieve different information about it. 

In [90]:
print(my_spart)
length = len(my_spart) # Define length
print('The length of my spart is ' + str(length))
sector = my_spart.sector() 
print('My spart is in the sector ' + str(sector))

[[3, 2], [4, 2, 2]]
The length of my spart is 5
My spart is in the sector (13, 2)


You can access the both $\Lambda^a$ and $\Lambda^s$

In [91]:
fermionic_part = my_spart[0]
print(fermionic_part)
bosonic_part = my_spart[1]
print(bosonic_part)

[3, 2]
[4, 2, 2]


#### `Superpartitions(n, m)`
Previously we have defined the whole space of superpartitions. Now let say that we want to deal with only on sector. Then, we just call the `Superpartitions` with `n` (the number of boxes, or bosonic degree) and `m` (the number of circles, or fermionic degree) as arguments:

In [92]:
spart_nm = Superpartitions(12, 3)
spart_nm

Superpartitions with 12 boxes and 3 circles

You can retrieve the elements of a set as a list, let's take a smaller set not to clutter the doccument. 

In [93]:
myset = Superpartitions(5, 2)
print(myset)
mylist = list(myset)
print(mylist)

Superpartitions with 5 boxes and 2 circles
[[[5, 0], []], [[4, 1], []], [[3, 2], []], [[4, 0], [1]], [[3, 1], [1]], [[3, 0], [2]], [[3, 0], [1, 1]], [[2, 1], [2]], [[2, 1], [1, 1]], [[2, 0], [3]], [[2, 0], [2, 1]], [[2, 0], [1, 1, 1]], [[1, 0], [4]], [[1, 0], [3, 1]], [[1, 0], [2, 2]], [[1, 0], [2, 1, 1]], [[1, 0], [1, 1, 1, 1]]]


Note that order in the list is deterministic but somewhat arbitrary. 

The set can also be used as an iterator:

In [94]:
an_other_set = Superpartitions(4,2)
print(an_other_set)
# We use it here as an iterator: 
for some_superpartition in an_other_set:
    print(some_superpartition)

Superpartitions with 4 boxes and 2 circles
[[4, 0], []]
[[3, 1], []]
[[3, 0], [1]]
[[2, 1], [1]]
[[2, 0], [2]]
[[2, 0], [1, 1]]
[[1, 0], [3]]
[[1, 0], [2, 1]]
[[1, 0], [1, 1, 1]]


### The $\Lambda^\ast, \Lambda^\circledast$ description
#### `partition_pair()`
We can obtain this description from a superpartition:

In [95]:
spart1 = S([[6,5,2,0],[4,3,3,2,1]])
print(spart1)
spart1_pair = spart1.partition_pair()
print(spart1_pair)

[[6, 5, 2, 0], [4, 3, 3, 2, 1]]
[[6, 5, 4, 3, 3, 2, 2, 1], [7, 6, 4, 3, 3, 3, 2, 1, 1]]


#### `Superpartitions.partition_pair_to_spart( a_partition_pair )`
We can also revert this operation (even though this operation is not really intended for end-user):

In [96]:
Superpartitions.partition_pair_to_spart(spart1_pair)

[[6, 5, 2, 0], [4, 3, 3, 2, 1]]

### Superdiagram
Not yet fully implemented. Latex support comming up. 

#### `terminal_diagram()`

We can obtain the representation of the superdiagram directely from a superpartition. 

In [97]:
S = Superpartitions()
a_spart = S([[7,5,2,0],[8,8,5,4,2,2]])
print(a_spart)
a_spart.terminal_diagram()

[[7, 5, 2, 0], [8, 8, 5, 4, 2, 2]]
■ ■ ■ ■ ■ ■ ■ ■
■ ■ ■ ■ ■ ■ ■ ■
■ ■ ■ ■ ■ ■ ■ ○
■ ■ ■ ■ ■ ○
■ ■ ■ ■ ■
■ ■ ■ ■
■ ■ ○
■ ■
■ ■
○



### Conjugate superpartition
#### `conjugate()` method
We can easily obtain the conjugate of a superpartition. 

In [98]:
S = Superpartitions()
a_spart = S([[9,7,5,2,0],[8,8,5,3,2,2,2,1]])
print(a_spart)
a_spart.terminal_diagram()
print('And now the conjugate superpartition')
a_spart_conjugate = a_spart.conjugate()
a_spart_conjugate.terminal_diagram()

[[9, 7, 5, 2, 0], [8, 8, 5, 3, 2, 2, 2, 1]]
■ ■ ■ ■ ■ ■ ■ ■ ■ ○
■ ■ ■ ■ ■ ■ ■ ■
■ ■ ■ ■ ■ ■ ■ ■
■ ■ ■ ■ ■ ■ ■ ○
■ ■ ■ ■ ■ ○
■ ■ ■ ■ ■
■ ■ ■
■ ■ ○
■ ■
■ ■
■ ■
■
○

And now the conjugate superpartition
■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ○
■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■
■ ■ ■ ■ ■ ■ ■ ○
■ ■ ■ ■ ■ ■
■ ■ ■ ■ ■ ■
■ ■ ■ ■ ○
■ ■ ■ ■
■ ■ ■ ○
■
○



### Dominance order

First, let's take three superpartitions with 10 boxes and 3 circles. 

In [99]:
S = Superpartitions(10,3)
spart_list = list(S)
spart1 = spart_list[22]
spart2 = spart_list[33]
spart3 = spart_list[35]

print('spart1')
print(spart1)
spart1.terminal_diagram()

print('spart2')
print(spart2)
spart2.terminal_diagram()

print('spart3')
print(spart3)
spart3.terminal_diagram()


spart1
[[5, 2, 1], [1, 1]]
■ ■ ■ ■ ■ ○
■ ■ ○
■ ○
■
■

spart2
[[4, 3, 0], [1, 1, 1]]
■ ■ ■ ■ ○
■ ■ ■ ○
■
■
■
○

spart3
[[4, 2, 1], [2, 1]]
■ ■ ■ ■ ○
■ ■ ○
■ ■
■ ○
■



#### `compare_dominance()` method
Let us now recover their ordering relations:

In [100]:
print('Compare spart1 and spart2')
print(S.compare_dominance(spart1, spart2))
print('Compare spart1 and spart3')
print(S.compare_dominance(spart1, spart3))
print('Compare spart2 and spart3')
print(S.compare_dominance(spart2, spart3))

Compare spart1 and spart2
>
Compare spart1 and spart3
>
Compare spart2 and spart3
Non-comparable


The `compare_dominance()` method returns one of the following strings: 
    `"=="`, 
    `">"`, 
    `"<"` or
    `"Non-comparable"`. We illustrate this with an example:

Let us consider the following set of superpartitions of sector $(4\,|\,1)$:

In [101]:
Sparts = Superpartitions(4,1)
print(Sparts)
print(list(Sparts))
for k in Sparts:
    print(k)

Superpartitions with 4 boxes and 1 circles
[[[4], []], [[3], [1]], [[2], [2]], [[2], [1, 1]], [[1], [3]], [[1], [2, 1]], [[1], [1, 1, 1]], [[0], [4]], [[0], [3, 1]], [[0], [2, 2]], [[0], [2, 1, 1]], [[0], [1, 1, 1, 1]]]
[[4], []]
[[3], [1]]
[[2], [2]]
[[2], [1, 1]]
[[1], [3]]
[[1], [2, 1]]
[[1], [1, 1, 1]]
[[0], [4]]
[[0], [3, 1]]
[[0], [2, 2]]
[[0], [2, 1, 1]]
[[0], [1, 1, 1, 1]]


We will compare the superpartitions $(2;2)$ to every superpartitions of the previous set:

In [102]:
Sparts = Superpartitions()
our_spart = Sparts([[2],[2]]) # We define (2; 2)
Sparts_4_1 = Superpartitions(4,1)
our_spart.terminal_diagram() # Print the diagram

for an_other_spart in Sparts_4_1:
    #We store the result of the operation in camparision
    comparision = Sparts.compare_dominance(our_spart, an_other_spart)
    if comparision == ">":
        print(str(our_spart) + " dominates " + str(an_other_spart))
    elif comparision == "==":
        print(str(our_spart) + " is exactly " + str(an_other_spart))
    elif comparision == "<":
        print(str(our_spart) + " is dominated by " + str(an_other_spart))
    elif comparision == "Non-comparable":
        print(str(our_spart) + " and " + str(an_other_spart) + " are non-comparable")

■ ■ ○
■ ■

[[2], [2]] is dominated by [[4], []]
[[2], [2]] is dominated by [[3], [1]]
[[2], [2]] is exactly [[2], [2]]
[[2], [2]] dominates [[2], [1, 1]]
[[2], [2]] is dominated by [[1], [3]]
[[2], [2]] dominates [[1], [2, 1]]
[[2], [2]] dominates [[1], [1, 1, 1]]
[[2], [2]] is dominated by [[0], [4]]
[[2], [2]] and [[0], [3, 1]] are non-comparable
[[2], [2]] dominates [[0], [2, 2]]
[[2], [2]] dominates [[0], [2, 1, 1]]
[[2], [2]] dominates [[0], [1, 1, 1, 1]]


Take note that the dominance ordering overloads the `>`, `<`, `>=`, ... operators. For instance

In [103]:
print( Sparts([[2],[2]]) <= Sparts([[3],[1]]))
print( Sparts([[2],[2]]) != Sparts([[2],[1,1]]))

True
True


Hence, the previous example can be implemented in a much more readeable fashion: 

In [104]:
Sparts = Superpartitions(4,1)
our_spart = Sparts([[2],[2]]) # We define (2; 2)

for an_other_spart in list(Sparts):
    if (our_spart > an_other_spart):
        print(str(our_spart) + " dominates " + str(an_other_spart))
    elif our_spart == an_other_spart:
        print(str(our_spart) + " is exactly " + str(an_other_spart))
    elif our_spart < an_other_spart:
        print(str(our_spart) + " is dominated by " + str(an_other_spart))
    else:
        print(str(our_spart) + " and " + str(an_other_spart) + " are non-comparable")

[[2], [2]] is dominated by [[4], []]
[[2], [2]] is dominated by [[3], [1]]
[[2], [2]] is exactly [[2], [2]]
[[2], [2]] dominates [[2], [1, 1]]
[[2], [2]] is dominated by [[1], [3]]
[[2], [2]] dominates [[1], [2, 1]]
[[2], [2]] dominates [[1], [1, 1, 1]]
[[2], [2]] is dominated by [[0], [4]]
[[2], [2]] and [[0], [3, 1]] are non-comparable
[[2], [2]] dominates [[0], [2, 2]]
[[2], [2]] dominates [[0], [2, 1, 1]]
[[2], [2]] dominates [[0], [1, 1, 1, 1]]


Smaller_spart
sort list
....

#### `get_smaller_sparts()` method
allows one to obtain the list of every superpartitions that is dominated by the one the method is called from. For instance,

In [105]:
Sparts = Superpartitions()
my_spart = Sparts([[2,0],[1,1]])
print("The superpartition")
my_spart.terminal_diagram()
print("dominates the following")
dominated_sparts = my_spart.get_smaller_sparts()
for spart in dominated_sparts:
    spart.terminal_diagram()

The superpartition
■ ■ ○
■
■
○

dominates the following
■ ■
■ ○
■
○

■ ○
■
■
■
○



#### `Superpartitions.give_greatest_spart( list_of_spart )`
this function allows one to exctract the greatest superpartition out of the list:

In [106]:
Sparts_4_2 = list(Superpartitions(4,2))
some_list = [Sparts_4_2[x] for x in [2,4,6,8]]
print(some_list)
print("The greatest element in the list is")
greatest = Superpartitions.give_greatest_spart(some_list)
print(greatest)

[[[3, 0], [1]], [[2, 0], [2]], [[1, 0], [3]], [[1, 0], [1, 1, 1]]]
The greatest element in the list is
[[3, 0], [1]]


#### `Superpartitions.sort_by_dominance( list_of_spart )`
allows to sort a list of superpartitions according to dominance. Non-comparable superpartitions are sorted in lexicographic order.

In [107]:
print(some_list)
sorted_list = Superpartitions.sort_by_dominance(some_list)
print("once sorted :")
print(sorted_list)

[[[3, 0], [1]], [[2, 0], [2]], [[1, 0], [3]], [[1, 0], [1, 1, 1]]]
once sorted :
[[[3, 0], [1]], [[1, 0], [3]], [[2, 0], [2]], [[1, 0], [1, 1, 1]]]


## Symmetric superpolynomials
We now introduce the space of symmetric superpolynomials. We first define the space:

In [108]:
Sym = SymSuperfunctionsAlgebra(QQ)
Sym

Symmetric superfunctions over Rational Field

We could have declared it with another coefficient ring, such as `ZZ`, but many things would fail because  their connection with the powersum basis (which is not a basis over  $\mathbb{Z}$).


### The monomial basis
Now this allows us to define the basis over the ring. For instance, one can introduce the monomial basis like so:

In [109]:
m = Sym.Monomial()
m

Symmetric superfunctions over Rational Field in the Monomial basis

We can now write an expression in the monomial basis:

In [110]:
Sparts = Superpartitions()
spart1 = Sparts([[3,2],[1,1]])
spart2 = Sparts([[3,0],[2,2]])
some_expr = 3*m(spart1) + 5/2*m(spart2)
print(some_expr)

3*m[[3, 2], [1, 1]] + 5/2*m[[3, 0], [2, 2]]


Now you see that we have defined two superpartition-type object to create our element of basis. When doing large scripts this is the way to go ( that is `m( superpartition )`). But when experimenting directly, it is rather cumbersome to define superpartitions, that's why abuse of notation is allowed. One could equally write the following:

In [111]:
easier_to_write = 3*m[[3,2],[1,1]] + 5/2*m[[3,0],[2,2]]
print(easier_to_write)

3*m[[3, 2], [1, 1]] + 5/2*m[[3, 0], [2, 2]]


Incidentally, this also allows copy-and-paste of printed results as valid sage expressions. Now back to the monomial basis, the product of two monomials is implemented:

In [112]:
my_pol = 3*m[[1,0],[3,2]] + 2/3*m[[2,0],[2,1,1]]
print(my_pol)
print('Result of left multiplication by m[[1],[]]')
my_other_pol = m[[1],[]]*my_pol
print(my_other_pol)

2/3*m[[2, 0], [2, 1, 1]] + 3*m[[1, 0], [3, 2]]
Result of left multiplication by m[[1],[]]
-2/3*m[[2, 1, 0], [2, 1, 1]] + 2/3*m[[3, 2, 0], [1, 1]] + 3*m[[3, 1, 0], [3]] + 3*m[[4, 1, 0], [2]]


As to the expansion of the monomial in the supercommutative variables, this should be implemented shortly. 

### Mutliplicative bases
We introduce the other three multiplicative basis respectively the power-sum, elementary and completely homogeneous bases:

In [113]:
p = Sym.Powersum()
e = Sym.Elementary()
h = Sym.Homogeneous()
print(p); print(e); print(h)

Symmetric superfunctions over Rational Field in the Powersum basis
Symmetric superfunctions over Rational Field in the Elementary basis
Symmetric superfunctions over Rational Field in the Homogeneous basis


To define elements of this basis, it works the same as with monomials. You can either declare `p( a_superpartition )` or write directely `p[[...], [...]]`

In [114]:
pol_in_p = 1/4*p[[1,0],[2,2]] + 3*p[[3,2], []]
pol_in_e = 1/4*e[[1,0],[2,2]] + 3*e[[3,2], []]
pol_in_h = 1/4*h[[1,0],[2,2]] + 3*h[[3,2], []]
print(pol_in_p)
print(pol_in_e)
print(pol_in_h)

1/4*p[[1, 0], [2, 2]] + 3*p[[3, 2], []]
1/4*e[[1, 0], [2, 2]] + 3*e[[3, 2], []]
1/4*h[[1, 0], [2, 2]] + 3*h[[3, 2], []]


The proper multiplication rules works out of the box:

In [115]:
print(p[[2],[]]*pol_in_p)
print(e[[2],[]]*pol_in_e)
print(h[[2],[]]*pol_in_h)

1/4*p[[2, 1, 0], [2, 2]]
1/4*e[[2, 1, 0], [2, 2]]
1/4*h[[2, 1, 0], [2, 2]]


It has to be noted that the basis indexed by the empty partition is considered to be unity (which differs from `p[[],[]] <-> 0` convention that we have in many formulas.)

In [116]:
print(p[[],[]]*pol_in_p)
print(e[[],[]]*pol_in_e)
print(h[[],[]]*pol_in_h)

1/4*p[[1, 0], [2, 2]] + 3*p[[3, 2], []]
1/4*e[[1, 0], [2, 2]] + 3*e[[3, 2], []]
1/4*h[[1, 0], [2, 2]] + 3*h[[3, 2], []]


### Change of basis
To obtain an expression on another basis is achieved by the following syntax:

`new_basis( expr )`

He are some examples :

In [117]:
my_expr = m[[2,0],[2]] + 1/2*m[[2,1],[1]]
print(my_expr)
my_expr_p = p(my_expr)
print(my_expr_p)

1/2*m[[2, 1], [1]] + m[[2, 0], [2]]
1/2*p[[2, 1], [1]] + p[[2, 0], [2]] - 1/2*p[[3, 1], []] - p[[4, 0], []]


Comparision between expressions on different basis is a valid operation:

In [118]:
print(my_expr == my_expr_p)

True


The implementation of change of basis is not lighting fast, but it seems sufficiently fast for most research purposes. 

Now the fact that we can compare two expressions expressed on different bases comes from the coercion model of Sage. The coercion is some kind of automatic conversion that allows one to make operations that make sense mathematically without having to explicitely program what is meant by this operation. For instance, you can multiply two different basis:

In [119]:
p[[1],[1,1]]*m[[3],[2,1]]

-p[[3, 1], [2, 1, 1, 1]] + p[[3, 1], [3, 1, 1]] + p[[4, 1], [2, 1, 1]] + p[[5, 1], [1, 1, 1]] - 2*p[[6, 1], [1, 1]]

So for example, we can test the relations (1.28-1.30) quite explicitely. Let us verify
$$
5 h_5 = \sum_{r=1}^5 p_r h_{5-r}
$$

In [120]:
# We first evaluate the right hand side (in python range(a,b) = [a,a+1, ..., b-1])
rhs = sum(p[[],[r]]*h[[],[5-r]] for r in range(1,6))
# We convert the sum to the basis h and print the result
print(h(rhs))

5*h[[], [5]]


Which was expected. We now evaluate
$$
6\tilde{h}_5 = \sum_{r=0}^5 p_r \tilde{h}_{n-r} + (r+1)\tilde{p}_rh_{n-r}
$$
Note that we split the sum in two since the program evaluates `p[[],[]]` as the identity instead of 0. 

In [121]:
sum1 = sum(p[[],[r]]*h[[5-r],[]] for r in range(1,6))  
sum2 = sum((r+1)*p[[r],[]]*h[[],[5-r]] for r in range(0,6))
print(h(sum1+sum2))

6*h[[5], []]


as expected. Finally, we evaluate the following
$$
(6+1)\tilde{e}_n = \sum_{r=0}^n (-1)^{r+1}\left[ p_r\tilde{e}_{6-r}
- (r+1)\tilde{p}_r e_{6-r} \right]
$$

In [122]:
sum1 = sum((-1)**(r+1)*p[[],[r]]*e[[6-r],[]] for r in range(1,7))  
sum2 = sum((-1)**(r+1)*(r+1)*p[[r],[]]*e[[],[6-r]] for r in range(0,7))
print(e(sum1-sum2))

7*e[[6], []]


which is again the expected result.  

### Ring homomorphism I
The $\hat{\omega}$ homomorphism is define on every basis. 
You can obtain the result of the involution with `expr.omega()`, it will return the effect of the homomorphism on the same basis you started from. Here is an exemple:

In [123]:
print('We have the following superpolynomial')
print(pol_in_e)
print('We apply the omega homomoprhism')
print(pol_in_e.omega())
print('We then convert the result to the homogeneous basis')
print(h(pol_in_e.omega()))

We have the following superpolynomial
1/4*e[[1, 0], [2, 2]] + 3*e[[3, 2], []]
We apply the omega homomoprhism
-13/4*e[[1, 0], [1, 1, 1, 1]] + 1/2*e[[1, 0], [2, 1, 1]] - 49/4*e[[1, 0], [2, 2]] + 6*e[[2, 0], [1, 1, 1]] - 3*e[[2, 1], [1, 1]] + 6*e[[2, 0], [2, 1]] - 6*e[[2, 1], [2]] + 12*e[[1, 0], [3, 1]] + 6*e[[3, 1], [1]] - 6*e[[2, 0], [3]] - 9*e[[3, 0], [1, 1]] + 6*e[[3, 0], [2]] - 3*e[[3, 2], []]
We then convert the result to the homogeneous basis
1/4*h[[1, 0], [2, 2]] + 3*h[[3, 2], []]


The operation is well defined on expression in any basis here is an example on the powersum basis:

In [124]:
an_expr = p[[2,0],[3,2]] + 1/2*p[[1,0],[3,2,1]]
print(an_expr)
print('Result of omega:')
an_expr.omega()

1/2*p[[1, 0], [3, 2, 1]] + p[[2, 0], [3, 2]]
Result of omega:


1/2*p[[1, 0], [3, 2, 1]] - p[[2, 0], [3, 2]]

### A scalar product
We now introduce the `expr.scalar_product(other_expr)` method. Here is an example of usage: 

In [125]:
Sparts_7_3 = Superpartitions(7,3)
sparts = list(Sparts_7_3)
print(len(sparts))
expr1 = p(sparts[0]) + 2*p(sparts[3]) + 5/2*p(sparts[6])
expr2 = p(sparts[0]) + 33*p(sparts[3]) + 13*p(sparts[5])
print(expr1)
print(expr2)
print('We now evaluate the scalar product between those two')
expr1.scalar_product(expr2)

19
5/2*p[[3, 2, 1], [1]] + 2*p[[4, 2, 1], []] + p[[6, 1, 0], []]
13*p[[4, 2, 0], [1]] + 33*p[[4, 2, 1], []] + p[[6, 1, 0], []]
We now evaluate the scalar product between those two


67

The method is well defined for any two basis. So for instance you can do the following:

In [126]:
expr_m = m[[3,0],[1,1]] + m[[3,1],[1]]+ m[[1,0],[1,1,1,1]]
expr_p = p[[5,0],[]] + p[[1,0],[2,1,1]] + 22*p[[1,0],[1,1,1,1]]
expr_m.scalar_product(expr_p)

23

Note that the implemented scalar product is always positive definite. The reason for this is that since 
$$ < p_\Lambda | p_\Lambda > = (-1)^{\binom{m}{2}}z_\Lambda $$
the sign is constant accros a single fermionic sector (indexed by the value of $m$).
Since we usualy deal with scalar products between elements of the same sector (that is, homogeneous superpolynomials) this has no incidence. 

We add examples illustrating some properties. First, the monomial and the homogeneous are dual with respect to the scalar product:

In [127]:
# We generate the set of superpartitions with 8 boxes and 3 circles
spart_set = Superpartitions(8,3)
# We take the scalar product between m_\Lambda and h_\Lambda for 
# every \Lambda in that set. The scalar product are store in a list.
scals = [m(spart).scalar_product(h(spart)) for spart in spart_set]
print(scals)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


It is known that the elementary basis is dual to the forgotten basis. 
Even though the later is not explicitely programmed, we can test that property by assuming $f_\Lambda = \hat{\omega}(m_\Lambda)$. 
Note that the following snipet of code might take a minute to execute. 
This is because, for every partition, the monomial has to be converted to powersums in order to apply the $\hat{\omega}$ involution and then converted back into monomials and back into powersum to evaluate the scalar product with $e_\Lambda$ which itself has to be converted to powersum in the process. 
If the forgotten basis is implemented in the future, this whole process should be much faster. 
For now though, the implementation of the forgotten basis is not a priority. 

In [128]:
# Generate some set
Sparts = Superpartitions(4,2)
scals = [m(spart).omega().scalar_product(e(spart)) for spart in spart_set]
print(scals)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### The one parameter space of symmetric superfunctions
We now want to define our superpolynomial ring over rational polynomials in one variable, namely $\alpha$. To do so, we have to introduce the ring like so

In [129]:
# We first define the coefficient ring
# Note that we add .fracion_field() so that it is allowed to 
# write expressions such as (alpha+1)/(3*alpha+2)
QQalpha = QQ['alpha'].fraction_field()
print(QQalpha)
# We can now introduce the symmetric functions
Sym_alpha = SymSuperfunctionsAlgebra(QQalpha)
print(Sym_alpha)
# We inject the basis definitions
Sym_alpha.inject_shorthands()

Fraction Field of Univariate Polynomial Ring in alpha over Rational Field
Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field
Defining m as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Monomial basis
Defining h as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Homogeneous basis
Defining p as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Powersum basis
Defining e as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Elementary basis


From there, it is now possible to define polynomials with coefficients including $\alpha$. Even though the ring knows the existence of the symbol $\alpha$, 
we must assign the symbol to a variable to directly write expressions with it:

In [130]:
alpha = QQalpha.gen()
# Now the variable alpha is associated with the symbol alpha
print(alpha)
# And so we can write expressions out of it
print(alpha**2 + 2*alpha)

alpha
alpha^2 + 2*alpha


And so, we are now able to introduce superpolynomials with coeffients as functions of $\alpha$: 

In [131]:
my_alpha_pol = m[[2,0],[1,1]] + 3*alpha/(2*alpha+2)*m[[1,0],[2,1]]
print(my_alpha_pol)

(3*alpha/(2*alpha+2))*m[[1, 0], [2, 1]] + m[[2, 0], [1, 1]]


### One paremeter deformation of the scalar product
Now that we have a proper ring, we can introduce a new scalar product. 

In [132]:
my_expr = p[[2,0],[3,1]] + p[[1,0],[3,1,1]]
my_expr.scalar_alpha(my_expr)

6*alpha^5 + 3*alpha^4

Note that again, this is well defined for any classical basis:

In [133]:
my_mono_expr = (2*(alpha + 1)*m[[2,0],[1,1]] +
                alpha**2*m[[1,0],[3]])
my_mono_expr.scalar_alpha(my_mono_expr)

3*alpha^7 + 4*alpha^6 + 14*alpha^5 + 30*alpha^4 + 26*alpha^3 + 8*alpha^2

### The $alpha$-deformed homogeneous basis
We now introduce the $g_\Lambda$ basis, it is introduced as `galpha`. 

So let us define, the space

In [134]:
QQa = QQ['alpha'].fraction_field()
Syma = SymSuperfunctionsAlgebra(QQa)
Syma.inject_shorthands()

Defining m as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Monomial basis
Defining h as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Homogeneous basis
Defining p as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Powersum basis
Defining e as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Elementary basis


We now define the `galpha` basis

In [135]:
galpha = Syma.Galpha()
galpha

Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Galpha basis

It works just like any other basis:

In [136]:
MyPol = galpha[[4],[1]]
print(MyPol)
MyPol_in_m = m(MyPol)
print(MyPol_in_m)

galpha[[4], [1]]
120/(24*alpha^6)*m[[0], [1, 1, 1, 1, 1]] + ((576*alpha+720)/(144*alpha^6))*m[[1], [1, 1, 1, 1]] + ((144*alpha+240)/(96*alpha^6))*m[[0], [2, 1, 1, 1]] + ((1152*alpha^2+4032*alpha+2880)/(1152*alpha^6))*m[[1], [2, 1, 1]] + ((3456*alpha^2+6336*alpha+2880)/(1152*alpha^6))*m[[2], [1, 1, 1]] + ((192*alpha^2+1152*alpha+960)/(768*alpha^6))*m[[0], [2, 2, 1]] + ((192*alpha^2+432*alpha+240)/(288*alpha^6))*m[[0], [3, 1, 1]] + ((11520*alpha^2+23040*alpha+11520)/(9216*alpha^6))*m[[1], [2, 2]] + ((18432*alpha^3+101376*alpha^2+129024*alpha+46080)/(36864*alpha^6))*m[[2], [2, 1]] + ((3456*alpha^3+17280*alpha^2+22464*alpha+8640)/(10368*alpha^6))*m[[1], [3, 1]] + ((1344*alpha^2+2304*alpha+960)/(2304*alpha^6))*m[[0], [3, 2]] + ((13824*alpha^3+32256*alpha^2+24192*alpha+5760)/(6912*alpha^6))*m[[3], [1, 1]] + ((276480*alpha^3+691200*alpha^2+552960*alpha+138240)/(331776*alpha^6))*m[[2], [3]] + ((2304*alpha^3+7296*alpha^2+6912*alpha+1920)/(9216*alpha^6))*m[[0], [4, 1]] + ((165888*alpha^3+340992*

Now we see here that the result of the conversion to the monomial basis looks rather messy. Sage does not provide a built-in way of factoring the coefficient of monomials, but one has been implemented. We easily obtain a more readable expression by typing `expr.collect()`: 

In [137]:
MyPol_in_m.collect()

5/alpha^6*m[[0], [1, 1, 1, 1, 1]] + ((4*alpha+5)/alpha^6)*m[[1], [1, 1, 1, 1]] + ((3*alpha+5)/(2*alpha^6))*m[[0], [2, 1, 1, 1]] + ((2*alpha^2+7*alpha+5)/(2*alpha^6))*m[[1], [2, 1, 1]] + ((5*alpha^2+10*alpha+5)/(4*alpha^6))*m[[1], [2, 2]] + ((6*alpha^2+11*alpha+5)/(2*alpha^6))*m[[2], [1, 1, 1]] + ((alpha^2+6*alpha+5)/(4*alpha^6))*m[[0], [2, 2, 1]] + ((2*alpha^3+11*alpha^2+14*alpha+5)/(4*alpha^6))*m[[2], [2, 1]] + ((4*alpha^2+9*alpha+5)/(6*alpha^6))*m[[0], [3, 1, 1]] + ((2*alpha^3+10*alpha^2+13*alpha+5)/(6*alpha^6))*m[[1], [3, 1]] + ((7*alpha^2+12*alpha+5)/(12*alpha^6))*m[[0], [3, 2]] + ((10*alpha^3+25*alpha^2+20*alpha+5)/(12*alpha^6))*m[[2], [3]] + ((12*alpha^3+28*alpha^2+21*alpha+5)/(6*alpha^6))*m[[3], [1, 1]] + ((6*alpha^3+19*alpha^2+18*alpha+5)/(24*alpha^6))*m[[0], [4, 1]] + ((18*alpha^3+37*alpha^2+24*alpha+5)/(12*alpha^6))*m[[3], [2]] + ((14*alpha^3+31*alpha^2+22*alpha+5)/(24*alpha^6))*m[[1], [4]] + ((24*alpha^4+74*alpha^3+79*alpha^2+34*alpha+5)/(24*alpha^6))*m[[4], [1]] + ((6*alpha

which is a little bit better. It not quite what we are looking for yet, but I will make progress on the presention front later. 

Now, let us illustrate the orthonormality property: 

In [138]:
sparts = Superpartitions(5,2)
scals = [m(spart).scalar_alpha(galpha(spart)) for spart in sparts]
print(scals)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Ring homomorphism II
We now introduce the $\alpha$-deformation of the endomorphism $\hat{\omega}$, namely $\hat{\omega}_\alpha$. To call it, you simply use the following formulation `expr.omega_alpha()`: 

In [139]:
my_expr = p[[1,0],[2,1,1,1]] + 32*p[[2,1],[3]]
omega_expr = my_expr.omega_alpha()
print(omega_expr)

alpha^6*p[[1, 0], [2, 1, 1, 1]] - 32*alpha^3*p[[2, 1], [3]]


To apply the operation, though, you have to make sure that you are indeed working with bases defined from the right coefficient ring, otherwise it will raise an error. 

In [140]:
bad_p = SymSuperfunctionsAlgebra(QQ).Powersum()
an_expr = bad_p[[1,0],[3]]
# We do the error handling here:
try:
    # It will run this 
    an_expr.omega_alpha()
    # and if it produces an error will execute instructions in except: 
except:
    print("An error has been raised")

An error has been raised


Now, since the $g_\Lambda$ basis is the $\alpha$-deformation of the homogeneous basis, we have the follwoing property
$$\hat{\omega}_\alpha(g_\Lambda) = e_\Lambda$$
As can be seen here: 

In [141]:
sparts = Superpartitions(4,2)
for spart in sparts:
    expr = galpha(spart)
    print(expr)
    print("We apply omega_alpha")
    om_expr = expr.omega_alpha()
    om_expr_in_e = e(om_expr)
    print(om_expr_in_e)
    print('')

galpha[[4, 0], []]
We apply omega_alpha
e[[4, 0], []]

galpha[[3, 1], []]
We apply omega_alpha
e[[3, 1], []]

galpha[[3, 0], [1]]
We apply omega_alpha
e[[3, 0], [1]]

galpha[[2, 1], [1]]
We apply omega_alpha
e[[2, 1], [1]]

galpha[[2, 0], [2]]
We apply omega_alpha
e[[2, 0], [2]]

galpha[[2, 0], [1, 1]]
We apply omega_alpha
e[[2, 0], [1, 1]]

galpha[[1, 0], [3]]
We apply omega_alpha
e[[1, 0], [3]]

galpha[[1, 0], [2, 1]]
We apply omega_alpha
e[[1, 0], [2, 1]]

galpha[[1, 0], [1, 1, 1]]
We apply omega_alpha
e[[1, 0], [1, 1, 1]]



and so the test is successful. 

# Jack superpolynomials
## Orthogonality characterization
Give the $\alpha$-scalar product and the dominance ordering on superpartition, one can define the Jack superpolynomials. 

We now introduce the implementation of those superfunctions. 

In [142]:
QQa = FractionField(QQ['alpha'])
alpha = QQa.gen()
Sym = SymSuperfunctionsAlgebra(QQa)
Sym.inject_shorthands()
Palpha = Sym.Jack()

Defining m as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Monomial basis
Defining h as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Homogeneous basis
Defining p as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Powersum basis
Defining e as shorthand for Symmetric superfunctions over Fraction Field of Univariate Polynomial Ring in alpha over Rational Field in the Elementary basis


This basis works like any other basis

In [143]:
my_expr = Palpha[[2,0],[3,2]] + Palpha[[3,0],[2,2]]
print(my_expr)

Palpha[[2, 0], [3, 2]] + Palpha[[3, 0], [2, 2]]


We know from the combinatorial definition that these superpolynomials are triangular on the monomial basis. We illustrate this fact by giving the expansion of the `Palpha`'s on the monomial basis for the sector $(5,3)$

In [144]:
for a_spart in Superpartitions(5,3):
    print("The Jack for " + str(a_spart) + " is ")
    print(m(Palpha(a_spart)))
    print('') #Spacing of output

The Jack for [[4, 1, 0], []] is 
(2/(2*alpha^2+3*alpha+1))*m[[2, 1, 0], [1, 1]] + (1/(2*alpha+1))*m[[2, 1, 0], [2]] + (2/(2*alpha+1))*m[[3, 1, 0], [1]] + (1/(2*alpha+1))*m[[3, 2, 0], []] + m[[4, 1, 0], []]

The Jack for [[3, 2, 0], []] is 
(1/(alpha^2+2*alpha+1))*m[[2, 1, 0], [1, 1]] + (-alpha/(2*alpha^2+4*alpha+2))*m[[2, 1, 0], [2]] + (1/(alpha+1))*m[[3, 1, 0], [1]] + m[[3, 2, 0], []]

The Jack for [[3, 1, 0], [1]] is 
(2/(alpha+1))*m[[2, 1, 0], [1, 1]] + (1/(alpha+1))*m[[2, 1, 0], [2]] + m[[3, 1, 0], [1]]

The Jack for [[2, 1, 0], [2]] is 
(2/(alpha+2))*m[[2, 1, 0], [1, 1]] + m[[2, 1, 0], [2]]

The Jack for [[2, 1, 0], [1, 1]] is 
m[[2, 1, 0], [1, 1]]



We also know that those polynomials are orthogonal:

In [145]:
sparts_set = Superpartitions(6,3)
sparts_pairs = Combinations(sparts_set, 2)
for a_pair in sparts_pairs:
    print(a_pair)
    spart1, spart2 = a_pair
    scalar_prod = Palpha(spart1).scalar_alpha(Palpha(spart2))
    print(scalar_prod)
    print('')

[[[5, 1, 0], []], [[4, 2, 0], []]]
0

[[[5, 1, 0], []], [[3, 2, 1], []]]
0

[[[5, 1, 0], []], [[4, 1, 0], [1]]]
0

[[[5, 1, 0], []], [[3, 2, 0], [1]]]
0

[[[5, 1, 0], []], [[3, 1, 0], [2]]]
0

[[[5, 1, 0], []], [[3, 1, 0], [1, 1]]]
0

[[[5, 1, 0], []], [[2, 1, 0], [3]]]
0

[[[5, 1, 0], []], [[2, 1, 0], [2, 1]]]
0

[[[5, 1, 0], []], [[2, 1, 0], [1, 1, 1]]]
0

[[[4, 2, 0], []], [[3, 2, 1], []]]
0

[[[4, 2, 0], []], [[4, 1, 0], [1]]]
0

[[[4, 2, 0], []], [[3, 2, 0], [1]]]
0

[[[4, 2, 0], []], [[3, 1, 0], [2]]]
0

[[[4, 2, 0], []], [[3, 1, 0], [1, 1]]]
0

[[[4, 2, 0], []], [[2, 1, 0], [3]]]
0

[[[4, 2, 0], []], [[2, 1, 0], [2, 1]]]
0

[[[4, 2, 0], []], [[2, 1, 0], [1, 1, 1]]]
0

[[[3, 2, 1], []], [[4, 1, 0], [1]]]
0

[[[3, 2, 1], []], [[3, 2, 0], [1]]]
0

[[[3, 2, 1], []], [[3, 1, 0], [2]]]
0

[[[3, 2, 1], []], [[3, 1, 0], [1, 1]]]
0

[[[3, 2, 1], []], [[2, 1, 0], [3]]]
0

[[[3, 2, 1], []], [[2, 1, 0], [2, 1]]]
0

[[[3, 2, 1], []], [[2, 1, 0], [1, 1, 1]]]
0

[[[4, 1, 0], [1]], [[3, 2, 0], 

## Normalization


## Evaluation

## Integral form of the Jack superpolynomials

Illustrate the conjecture

## Duality
We illustrate the duality property.

We first pick a superpartition and obtain the Jack superpolynomial associated with it.

In [146]:
sparts = Superpartitions()
spart1 = sparts([[4,1],[]])
spart1.terminal_diagram()
a_Jack = Palpha(spart1)
print(a_Jack)

■ ■ ■ ■ ○
■ ○

Palpha[[4, 1], []]


We compute the norm of this polynomial and apply $\hat{\omega}_\alpha$ on the Jack. We then divide the result by the norm and multiply by the appropriate sign.

In [147]:
the_norm = a_Jack.scalar_alpha(a_Jack)
omega_a_Jack = a_Jack.omega_alpha()
normalized_om_Jack = (-1)**(1)*omega_a_Jack/the_norm
print(normalized_om_Jack.collect())

((12*alpha^2-12)/(4*alpha^2+17*alpha+4))*Palpha[[1, 0], [1, 1, 1, 1]] + Palpha[[1, 0], [2, 1, 1]]


Now, according to the duality property, this polynomial should be $P^{1/\alpha}_{(1,0;2,1,1)}$. To check this, we develop this polynomial on the monomial basis so that no $\alpha$ is hidden inside a definition and substitute $\alpha$ for $1/\alpha$. Converting back to the Jack basis, we should find $P^\alpha_{(1,0;2,1,1)}$

In [148]:
m_expr = m(normalized_om_Jack)
invert_alpha_m = m_expr.subs_coeff({alpha:1/alpha})
resulting_jack = Palpha(invert_alpha_m)
print(resulting_jack)

Palpha[[1, 0], [2, 1, 1]]


Which is indeed the expected result. 

## Particular expressions

P vs g

In [149]:
galpha(Palpha[[],[5]]).collect()

(120*alpha^5/(24*alpha^4+50*alpha^3+35*alpha^2+10*alpha+1))*galpha[[], [5]]

In [150]:
galpha(Palpha[[5],[]]).collect()

(120*alpha^6/(120*alpha^5+274*alpha^4+225*alpha^3+85*alpha^2+15*alpha+1))*galpha[[5], []]

P vs m, P vs e

In [151]:
aJack = Palpha[[2,0],[3,1]]
print(aJack)
aJack_m = m(aJack)
lim_aJack = aJack_m.map_coefficients(
    lambda x: QQa(SR(x).limit(alpha = infinity)))
print(lim_aJack)

Palpha[[2, 0], [3, 1]]
m[[2, 0], [3, 1]]


In [152]:
alpha_zero_Jack = aJack_m.map_coefficients(lambda x: x.subs({alpha:0}))
print(e(alpha_zero_Jack))

-e[[3, 1], [2]]


P vs h

In [153]:
nJack_m = m(Palpha[[],[5]])
alpha_one = nJack_m.subs_coeff({alpha:1})
print(h(alpha_one))

h[[], [5]]


In [154]:
nJack_m = m(Palpha[[5],[]])
alpha_one = nJack_m.map_coefficients(lambda x: x.subs({alpha:1}))
print(h(alpha_one))

1/6*h[[5], []]


## negative fractional parameter

# Macdonald superpolynomials

In [155]:
QQqt = FractionField(QQ['q', 't'])
q, t = QQqt.gens()
symqt = SymSuperfunctionsAlgebra(QQqt)
symqt.inject_shorthands()
Pqt = symqt.Macdonald()

Defining m as shorthand for Symmetric superfunctions over Fraction Field of Multivariate Polynomial Ring in q, t over Rational Field in the Monomial basis
Defining h as shorthand for Symmetric superfunctions over Fraction Field of Multivariate Polynomial Ring in q, t over Rational Field in the Homogeneous basis
Defining p as shorthand for Symmetric superfunctions over Fraction Field of Multivariate Polynomial Ring in q, t over Rational Field in the Powersum basis
Defining e as shorthand for Symmetric superfunctions over Fraction Field of Multivariate Polynomial Ring in q, t over Rational Field in the Elementary basis


In [156]:
Pqt[[4,1],[3,2]]

Pqt[[4, 1], [3, 2]]

## Particular cases
$q=t^\alpha, t\rightarrow 1$


$q=0, q \rightarrow \infty$

$t=1$

$q=1$

$q=t$ See Schur polynomial

### $g_\Lambda(q,t)$

## Macdonald superpolynomials from an eigenvalue problem
This is a bit more complicated and will have to wait. 

## Further properties 
### One part superpartition

### Duality

In [157]:
spart1 = Superpartitions()([[2,0],[1,1]])
a_macdo = Pqt(spart1)
print(a_macdo)
spart1.terminal_diagram()
w_a_macdo = a_macdo.omega_qt()
new_mac = (t/q)**spart1.bosonic_degree()*w_a_macdo
new_mac_m = m(new_mac)
new_mac_m = new_mac_m.subs_coeff({q:1/t, t:1/q})
new_mac = Pqt(new_mac_m)
print(new_mac.collect())
new_mac.monomial_coefficients().keys()[0].terminal_diagram()

Pqt[[2, 0], [1, 1]]
■ ■ ○
■
■
○

((-q^2*t^2+q^2*t+t-1)/(q^6-q^5-q^4+q^3))*Pqt[[3, 0], [1]]
■ ■ ■ ○
■
○



### Normalization

### Specialization

### Symmetry

Conjecture

### Positivity

Conjecture

### Antoher scalar product

## Constructing the Macdonald superpolynomials from the non-symmetric Macdonald polynomials

### Cherednik algebra

### The non-symmetric Macdonald polynomials

# The double Macdonald polynomials